In [ ]:
#pip install azure-mgmt-resource
#pip install azure-mgmt-datalake-store
#pip install azure-datalake-storr

In [ ]:
pip install pandas_datareader

In [ ]:
pip install pandas
pip install azure
pip install numpy
pip install urllib

In [ ]:
pip install azure-storage==0.36.0
#pip install azure-storage-blob==0.37.1

In [ ]:
pip freeze

In [ ]:
## Milestone 1

In [1]:
from urllib.request import Request
import urllib
from datetime import datetime
import numpy as np
import pandas as pd
import csv
import os

In [2]:
##Pre-defined
import os, uuid, sys
from azure.storage.blob import BlockBlobService, PublicAccess
block_blob_service = BlockBlobService(account_name='wqd7005', account_key='XpndfNFVKvJKGU22QzDYnw8VKc91G4QIW3JESEZQSO4hxLkCB16O+F0wdo6IAAuPvJ+FizhoDO7AWKm4x9N6xw==')
container_name ='rawblob'

## Download blob in dataframe
from io import StringIO
import pandas as pd
generator = block_blob_service.list_blobs(container_name)
tables = {}
for blob in generator:
    local_file_name=os.path.join(blob.name)
    #from azure.storage.blob import BlobService
    #print(local_file_name)
    blobstring = block_blob_service.get_blob_to_text(container_name,local_file_name).content
    #df = pd.read_csv(StringIO(blobstring))
    names1 = blob.name[5:]
    names = names1[:-4]
    print(names)
    tables[names] = pd.read_csv(StringIO(blobstring))

ADVENTA
AHEALTH
DPHARMA
HARTA
IHH
KOSSAN
KOTRA
KPJ
PHARMA
SUPERMX
TMCLIFE
TOPGLOV
YSPSAH


In [3]:
for name in list(tables):
    tables[name].iloc[:,0] = pd.to_datetime(tables[name].iloc[:,0], unit='s').dt.strftime('%Y-%m-%d')
    #print(name)
    #print(tables[name].iloc[:,0])

## Use counter KOSSAN to create model

In [8]:
df=tables['KOSSAN']

In [9]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas_datareader.data as web
style.use('ggplot')
df=tables['KOSSAN']
df['100ma']=df['c'].rolling(window=100,min_periods=0).mean()
df['20ma']=df['c'].rolling(window=20,min_periods=0).mean()
df.dropna()

D:\software\anaconda\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


,t,c,o,h,l,v,s,100ma,20ma
0,2009-01-01,0.723,0.698,0.723,0.698,357600,ok,0.723000,0.723000
1,2009-01-04,0.753,0.726,0.753,0.726,118300,ok,0.738000,0.738000
2,2009-01-05,0.748,0.758,0.783,0.743,430500,ok,0.741333,0.741333
3,2009-01-06,0.768,0.758,0.778,0.758,160200,ok,0.748000,0.748000
4,2009-01-07,0.763,0.763,0.763,0.746,122400,ok,0.751000,0.751000
...,...,...,...,...,...,...,...,...,...
2691,2019-12-23,4.170,4.180,4.180,4.160,45900,ok,4.188300,4.153000
2692,2019-12-25,4.200,4.160,4.250,4.160,312700,ok,4.189600,4.153000
2693,2019-12-26,4.250,4.180,4.250,4.160,311200,ok,4.191400,4.155500
2694,2019-12-29,4.200,4.240,4.240,4.190,467800,ok,4.192700,4.156000


In [29]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
get_ipython().magic('matplotlib qt')
ax1=plt.subplot2grid((13,2),(0,0),rowspan=10, colspan=2)
ax2=plt.subplot2grid((13,2),(11,0),rowspan=2, colspan=2, sharex=ax1)

ax1.plot(df['t'], df['c'], label="Closing Price")
ax1.plot(df['100ma'], label="100ma")
ax1.plot(df['20ma'], label="20ma")
ax1.legend(loc='upper left', frameon=False)

ax2.bar(df.index, df['v'])
ax2.set_ylabel('Volume', fontsize=8)

#set ticks every week
ax1.xaxis.set_major_locator(mdates.MonthLocator())
#set major ticks format
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))



plt.show()

## Using Regression

In [11]:
#For Prediction
## Regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, svm
df['prediction'] = df['c'].shift(-1)
forecast_time = int(len(df))
df =df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [12]:
X = np.array(df['c'])
Y = np.array(df['prediction'])
X = preprocessing.scale(X)
X_prediction = X[-forecast_time:]
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, test_size=0.5)

X_train= X_train.reshape(-1, 1)
Y_train= Y_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
Y_test = Y_test.reshape(-1, 1)
X_prediction = X_prediction.reshape(-1, 1)

In [13]:
#Performing the Regression on the training data
clf = LinearRegression()
clf.fit(X_train, Y_train)
confidence = clf.score(X_test, Y_test)
prediction = (clf.predict(X_prediction))

In [25]:
print("Confidence Level:", confidence)

Confidence Level: 0.9966730015252258


In [14]:
df['predict2'] = prediction

D:\software\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
df

,t,c,o,h,l,v,s,100ma,20ma,prediction,predict2
0,2009-01-01,0.723,0.698,0.723,0.698,357600,ok,0.723000,0.723000,0.753,0.727045
1,2009-01-04,0.753,0.726,0.753,0.726,118300,ok,0.738000,0.738000,0.748,0.757031
2,2009-01-05,0.748,0.758,0.783,0.743,430500,ok,0.741333,0.741333,0.768,0.752034
3,2009-01-06,0.768,0.758,0.778,0.758,160200,ok,0.748000,0.748000,0.763,0.772024
4,2009-01-07,0.763,0.763,0.763,0.746,122400,ok,0.751000,0.751000,0.793,0.767027
...,...,...,...,...,...,...,...,...,...,...,...
2690,2019-12-22,4.160,4.180,4.210,4.160,332100,ok,4.187400,4.156000,4.170,4.162454
2691,2019-12-23,4.170,4.180,4.180,4.160,45900,ok,4.188300,4.153000,4.200,4.172449
2692,2019-12-25,4.200,4.160,4.250,4.160,312700,ok,4.189600,4.153000,4.250,4.202435
2693,2019-12-26,4.250,4.180,4.250,4.160,311200,ok,4.191400,4.155500,4.200,4.252412


In [30]:
import matplotlib.dates as mdates
get_ipython().magic('matplotlib qt')
ax1=plt.subplot2grid((13,2),(0,0),rowspan=10, colspan=2)
ax2=plt.subplot2grid((13,2),(11,0),rowspan=2, colspan=2, sharex=ax1)

ax1.plot(df['t'], df['c'], label="Closing Price")
ax1.plot(df['predict2'], '-b', label="Prediction")
ax1.plot(df['20ma'], '--r', label="20ma")
ax1.legend(loc='upper left', frameon=False)

ax2.bar(df.index, df['v'])
ax2.set_ylabel('Volume', fontsize=8)

#set ticks every week
ax1.xaxis.set_major_locator(mdates.MonthLocator())
#set major ticks format
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))



plt.show()

## Using LSTM

In [22]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

#creating dataframe
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['t', 'c'])
for i in range(0,len(data)):
    new_data['t'][i] = data['t'][i]
    new_data['c'][i] = data['c'][i]

#setting index
new_data.index = new_data.t
new_data.drop('t', axis=1, inplace=True)

#creating train and test sets
dataset = new_data.values

train = dataset[0:2500,:]
valid = dataset[2500:,:]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

#predicting 246 values, using past 60 from the train data
inputs = new_data['c'].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

Using TensorFlow backend.


Epoch 1/1
 - 100s - loss: 0.0018


In [31]:
plt.figure(figsize = (18,9))
plt.plot(range(len(df.loc[60:,'c'])),df.loc[60:,'c'],color='b',label='True')
plt.plot(range(len(df.loc[60:,'c'])),closing_price,color='orange',label='Prediction')
#plt.xticks(range(0,df.shape[0],50),df['Date'].loc[::50],rotation=45)
plt.xlabel('Date')
plt.ylabel('Mid Price')
plt.legend(fontsize=18)
plt.show()